In [1]:
### This file will create a csv file with the following:

# Each polling place will have the amount of votes cast 
# Each polling place casts n votes in an SA1
# Use that to predict the amount of votes cast in each SA1
# End file should read
# SA1, ALP_primary, LNP_primary, GRN_primary, PHON_primary, UAP_primary, IND_primary, OTH_primary, ALP_tpp, INFORMAL, tot_population

import os
path = os.getcwd()
import pandas as pd
import numpy as np

In [2]:
# Input the data for primaries
raw_primaries = pd.read_csv(f"{path}/data/HouseStateFirstPrefsByPollingPlaceDownload-27966-NSW.csv",header=1)
raw_primaries.iloc[0:9]

# Drop the unwanted columns
handled_primaries = raw_primaries.fillna("oth") # Clean up the empty values
# for index, row in handled_primaries.iterrows():
#     if row["PartyNm"] == "Independent":
#         handled_primaries.at[index,"PartyNm"] = f"Independent{row['Surname']}"
handled_primaries = handled_primaries[["DivisionNm","PollingPlaceID","PartyNm","OrdinaryVotes"]]
handled_primaries.iloc[0:9]
# Combine to get votes for each polling place
handled_primaries = handled_primaries.pivot_table(index=["PollingPlaceID","DivisionNm"],columns="PartyNm",values="OrdinaryVotes",aggfunc="sum")
# Handle microparties -> Only keep the columns listed, everything else is "Others"
columns_to_keep = ['Informal','Labor', 'Liberal',"Pauline Hanson's One Nation",'The Greens', 'The Nationals', 'United Australia Party','Independent']
handled_primaries['Others'] = handled_primaries.drop(columns_to_keep, axis=1).sum(axis=1)
filtered_primaries = handled_primaries[columns_to_keep + ['Others']]
# Set the Nan values to 0
filtered_primaries = filtered_primaries.fillna(0)
filtered_primaries.iloc[0:9]

,PartyNm,Informal,Labor,Liberal,Pauline Hanson's One Nation,The Greens,The Nationals,United Australia Party,Independent,Others
PollingPlaceID,DivisionNm,,,,,,,,,
1,Watson,230.0,810.0,473.0,111.0,119.0,0.0,121.0,0.0,0.0
2,Banks,63.0,356.0,444.0,31.0,94.0,0.0,51.0,0.0,22.0
3,Banks,132.0,427.0,404.0,32.0,65.0,0.0,67.0,0.0,35.0
4,Banks,139.0,529.0,1127.0,52.0,158.0,0.0,110.0,0.0,62.0
5,Banks,109.0,597.0,712.0,52.0,166.0,0.0,137.0,0.0,42.0
6,Banks,226.0,820.0,827.0,51.0,205.0,0.0,101.0,0.0,42.0
7,Banks,178.0,647.0,783.0,48.0,158.0,0.0,121.0,0.0,55.0
8,Banks,145.0,421.0,533.0,30.0,108.0,0.0,97.0,0.0,30.0
10,Banks,165.0,552.0,925.0,66.0,134.0,0.0,133.0,0.0,44.0


In [3]:
# Input the data for primaries
raw_primaries = pd.read_csv(f"{path}/data/HouseStateFirstPrefsByPollingPlaceDownload-27966-NSW.csv",header=1)
raw_primaries.iloc[0:9]

# Drop the unwanted columns
handled_primaries = raw_primaries.fillna("oth") # Clean up the empty values
# for index, row in handled_primaries.iterrows():
#     if row["PartyNm"] == "Independent":
#         handled_primaries.at[index,"PartyNm"] = f"Independent{row['Surname']}"
handled_primaries = handled_primaries[["DivisionNm","PollingPlaceID","PartyNm","OrdinaryVotes"]]
handled_primaries.iloc[0:9]
# Combine to get votes for each polling place
handled_primaries = handled_primaries.pivot_table(index=["PollingPlaceID","DivisionNm"],columns="PartyNm",values="OrdinaryVotes",aggfunc="sum")
# Handle microparties -> Only keep the columns listed, everything else is "Others"
columns_to_keep = ['Informal','Labor', 'Liberal',"Pauline Hanson's One Nation",'The Greens', 'The Nationals', 'United Australia Party','Independent']
handled_primaries['Others'] = handled_primaries.drop(columns_to_keep, axis=1).sum(axis=1)
filtered_primaries = handled_primaries[columns_to_keep + ['Others']]
# Set the Nan values to 0
filtered_primaries = filtered_primaries.fillna(0)

# Input the data for tpp
raw_tpp = pd.read_csv(f"{path}/data/HouseTppByPollingPlaceDownload-27966.csv",header=1)
# Remove any rows that are not for NSW
raw_nsw_tpp = raw_tpp[raw_tpp["StateAb"]=="NSW"]

# Drop the unwanted columns
handled_tpp = raw_nsw_tpp[["PollingPlaceID","DivisionNm","Liberal/National Coalition Votes","Australian Labor Party Votes"]]
# Reset index to DivisionNm and PollingPlaceID
handled_tpp = handled_tpp.set_index(["PollingPlaceID","DivisionNm",])
# Rename columns to LNP_tpp and ALP_tpp
handled_tpp.columns = ["LNP_tpp","ALP_tpp"]
# Merge the two dataframes
votes_by_polling_id = pd.merge(filtered_primaries,handled_tpp,on=["PollingPlaceID","DivisionNm"])

In [4]:
votes_by_polling_id

,,Informal,Labor,Liberal,Pauline Hanson's One Nation,The Greens,The Nationals,United Australia Party,Independent,Others,LNP_tpp,ALP_tpp
PollingPlaceID,DivisionNm,,,,,,,,,,,
1,Watson,230.0,810.0,473.0,111.0,119.0,0.0,121.0,0.0,0.0,623,1011
2,Banks,63.0,356.0,444.0,31.0,94.0,0.0,51.0,0.0,22.0,524,474
3,Banks,132.0,427.0,404.0,32.0,65.0,0.0,67.0,0.0,35.0,488,542
4,Banks,139.0,529.0,1127.0,52.0,158.0,0.0,110.0,0.0,62.0,1289,749
5,Banks,109.0,597.0,712.0,52.0,166.0,0.0,137.0,0.0,42.0,876,830
...,...,...,...,...,...,...,...,...,...,...,...,...
108639,Wentworth,0.0,30.0,105.0,2.0,27.0,0.0,1.0,105.0,4.0,151,123
108651,Chifley,4.0,202.0,79.0,25.0,32.0,0.0,22.0,11.0,11.0,122,260
108659,Calare,5.0,69.0,0.0,23.0,23.0,139.0,12.0,80.0,0.0,196,150


In [5]:
def swap_to(df,index_name):
    if index_name == 'DivisionNm':
        return votes_by_polling_id.reset_index().set_index(['DivisionNm','PollingPlaceID']).sort_index()
    elif index_name == 'PollingPlaceID':
        return votes_by_polling_id.reset_index().set_index(['PollingPlaceID','DivisionNm']).sort_index()
    else:
        raise ValueError("index_name must be either 'DivisionNm' or 'PollingPlaceID'")
        return None

In [6]:
# Input the SA1 data
raw_sa1 = pd.read_csv(f"{path}/data/2022-federal-election-votes-sa1.csv",header=0)
# Remove any rows that are not for NSW
raw_nsw_sa1 = raw_sa1[raw_sa1["state_ab"]=="NSW"]
# Remove any rows with pp_id = 0
raw_nsw_sa1 = raw_nsw_sa1[raw_nsw_sa1["pp_id"]!=0]
handled_sa1 = raw_nsw_sa1[["ccd_id","pp_id","votes"]]
handled_sa1.reset_index(drop=True,inplace=True)
handled_sa1 = handled_sa1.pivot_table(index=["ccd_id"],columns="pp_id",values="votes",aggfunc="sum")

In [7]:
# Create a new table with SA1s as the index and same columns as the merged table
sa1_votes = pd.DataFrame(index=handled_sa1.index,columns=votes_by_polling_id.columns)
# Enforce index to be PollingPlaceID
votes_by_polling_id = swap_to(votes_by_polling_id,'PollingPlaceID')

for ccd_id, _ in sa1_votes.iterrows():
    # Get the polling places in the SA1
    pp_ids = handled_sa1.loc[ccd_id].dropna()
    # Get the votes for each polling place
    # sa1_votes[ccd_id] = weighted average of the votes for each polling place
    row = pd.Series(index=sa1_votes.columns,dtype=np.float64)

    # row.name = pp_ids.iloc[0].name
    row.fillna(0,inplace=True)
    for pp_id, total_votes in pp_ids.items():
        pp_result = votes_by_polling_id.loc[pp_id].iloc[0] # .iloc removes DivisionNm in this context
        row += pp_result/sum(pp_result.values) * total_votes
        row_name = pp_result.name
    row.name = row_name
    sa1_votes.loc[ccd_id] = row

In [8]:
sa1_votes.head()

,Informal,Labor,Liberal,Pauline Hanson's One Nation,The Greens,The Nationals,United Australia Party,Independent,Others,LNP_tpp,ALP_tpp
ccd_id,,,,,,,,,,,
1059999,0.062369,0.225181,0.144359,0.036907,0.025228,0.0,0.037141,0.0,0.0,0.196216,0.2726
1100701,6.099145,13.662768,27.466835,4.18587,3.875221,0.0,1.786152,19.079763,3.893817,38.526592,35.423835
1100702,5.895406,41.305792,26.505289,4.203689,11.289922,0.0,1.828803,1.903916,4.514887,33.078631,58.473666
1100703,6.813759,47.979636,30.923625,4.874166,13.139519,0.0,2.145193,2.238714,5.292267,38.663676,67.929444
1100704,9.426265,64.837377,39.264463,6.269449,18.622192,0.0,2.622494,2.825366,7.345526,49.049422,92.737446


In [9]:
sa1_votes.shape

(17997, 11)

In [10]:
# import geopandas as gpd
# shape = gpd.read_file(f"{path}/data/SA1_2021_AUST_SHP_GDA2020/SA1_2021_AUST_GDA2020.shp")
# shape = shape[~shape["geometry"].isnull()] # Remove any rows with null geometry
# shape = shape[shape["STE_NAME21"]=="New South Wales"]

# # Rename SA1_CODE21 to ccd_id
# # shape.set_index("SA1_CODE21",inplace=True)
# # SA1_CODE21 take the first 1 digit and last 6 for ccd_id
# shape["ccd_id"] = shape["SA1_CODE21"].apply(lambda x: int(str(x)[0] + str(x)[-6:]))
# shape.set_index("ccd_id",inplace=True)

# print(shape.columns)
# new_shape = shape[["geometry"]]
# # Merge the two tables
# final_table = pd.merge(sa1_votes,shape,on="ccd_id")
# # Convert to geodataframe
# final_table = gpd.GeoDataFrame(final_table,geometry="geometry")

In [10]:
import geopandas as gpd
shape = gpd.read_file(f"{path}/data/1270055001_sa1_2016_aust_shape/SA1_2016_AUST.shp")

In [11]:
shape = shape[~shape["geometry"].isnull()] # Remove any rows with null geometry
shape = shape[shape["STE_NAME16"]=="New South Wales"]

# Rename SA1_CODE21 to ccd_id
# shape.set_index("SA1_CODE21",inplace=True)

In [12]:
shape.head()

,SA1_MAIN16,SA1_7DIG16,SA2_MAIN16,SA2_5DIG16,SA2_NAME16,SA3_CODE16,SA3_NAME16,SA4_CODE16,SA4_NAME16,GCC_CODE16,GCC_NAME16,STE_CODE16,STE_NAME16,AREASQKM16,geometry
0,10102100701,1100701,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,362.8727,"POLYGON ((149.71174 -35.12318, 149.71184 -35.1..."
1,10102100702,1100702,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,229.7459,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3..."
2,10102100703,1100703,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,2.3910,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4..."
3,10102100704,1100704,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,1.2816,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4..."
4,10102100705,1100705,101021007,11007,Braidwood,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,1.1978,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4..."


In [34]:
sa1_pop = pd.read_csv(f"{path}/data/2016_GCP_SA1_for_NSW_short-header/2016 Census GCP Statistical Area 1 for NSW/2016Census_G01_NSW_SA1.csv",header=0)
sa1_pop = sa1_pop[["SA1_7DIGITCODE_2016","Tot_P_P"]]
sa1_pop.columns = ["SA1_7DIG16","pop"]

# sa1_pop.set_index("SA1_7DIGITCODE_2016",inplace=True)

In [36]:
# Add a column to shape for population
shape["pop"] = sa1_pop["pop"]

In [41]:
newshape = shape[["SA1_7DIG16","SA3_NAME16","pop","geometry"]]
newshape.columns = ["ccd_id","Name","Population","geometry"]
newshape.set_index("ccd_id",inplace=True)
# Merge the two tables if sa1_votes doesn't have ccd_id as index set default values to 0
# Set index to integers for both tables
# Round sa1_votes to 2 decimal places
sa1_votes = sa1_votes.astype(float).round(decimals=2)
sa1_votes.index = sa1_votes.index.astype(int)
newshape.index = newshape.index.astype(int)

final_table = pd.merge(newshape,sa1_votes,on="ccd_id",how='inner')
final_table.fillna(0,inplace=True)
# Convert to geodataframe
final_table = gpd.GeoDataFrame(final_table,geometry="geometry")

# Add a column called DivisionNm with default value empty string



In [42]:
final_table.head()

,Name,Population,geometry,Informal,Labor,Liberal,Pauline Hanson's One Nation,The Greens,The Nationals,United Australia Party,Independent,Others,LNP_tpp,ALP_tpp
ccd_id,,,,,,,,,,,,,,
1100701,Queanbeyan,256,"POLYGON ((149.71174 -35.12318, 149.71184 -35.1...",6.10,13.66,27.47,4.19,3.88,0.0,1.79,19.08,3.89,38.53,35.42
1100702,Queanbeyan,381,"POLYGON ((149.73421 -35.36758, 149.74475 -35.3...",5.90,41.31,26.51,4.20,11.29,0.0,1.83,1.90,4.51,33.08,58.47
1100703,Queanbeyan,428,"POLYGON ((149.77998 -35.44067, 149.77996 -35.4...",6.81,47.98,30.92,4.87,13.14,0.0,2.15,2.24,5.29,38.66,67.93
1100704,Queanbeyan,446,"POLYGON ((149.79744 -35.44149, 149.79778 -35.4...",9.43,64.84,39.26,6.27,18.62,0.0,2.62,2.83,7.35,49.05,92.74
1100705,Queanbeyan,402,"POLYGON ((149.79498 -35.44902, 149.79517 -35.4...",7.16,48.59,27.07,4.37,14.49,0.0,1.66,1.91,5.83,33.86,70.06


In [43]:
pp_id_hash = dict(votes_by_polling_id.index)
getdivision = lambda ccd_id: pp_id_hash[pd.Series.idxmax(handled_sa1.loc[ccd_id].dropna())]

# Adding a column called DivisionNm to final_table with value computed using getdivision
final_table["DivisionNm"] = final_table.index.map(getdivision)
# Reorder columns
final_table = final_table[["DivisionNm","Name","Population","ALP_tpp","Informal","Labor","Liberal","Pauline Hanson's One Nation","The Greens","The Nationals","United Australia Party","Independent","Others","LNP_tpp","geometry",]]


In [63]:
# Get all unique elements in DivisionNm
divisions = final_table["DivisionNm"].unique()

# Create a new table with DivisionNm as index and same columns as final_table
div_votes = pd.DataFrame(index=divisions,columns=final_table.columns)
# Drop ["DivisionNm","Name","geometry"] columns
div_votes.drop(["DivisionNm","Name","geometry"],axis=1,inplace=True)
div_votes.index.name = "DivisionNm"

# For each division, sum the votes of all SA1s in that division
for division in divisions:
    div_votes.loc[division] = final_table[final_table["DivisionNm"]==division].sum()
div_votes = div_votes.astype(int).round()


C:\Users\patri\AppData\Local\Temp\ipykernel_31120\2117862787.py:12: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  div_votes.loc[division] = final_table[final_table["DivisionNm"]==division].sum()


In [64]:
# Convert index to column and set population as the index
div_votes.reset_index(inplace=True)
div_votes.set_index("Population",inplace=True)
# Order index by size
div_votes.sort_index(inplace=True)

In [65]:
div_votes

,DivisionNm,ALP_tpp,Informal,Labor,Liberal,Pauline Hanson's One Nation,The Greens,The Nationals,United Australia Party,Independent,Others,LNP_tpp
Population,,,,,,,,,,,,
130193,Newcastle,30728,2659,19798,10794,1996,9261,0,1059,0,1971,14153
134724,Shortland,23681,2902,17002,13542,2703,4254,6,1331,1056,2473,18690
139715,Dobell,24666,2207,18798,14457,3267,3675,0,1639,0,1519,18692
140040,Cunningham,29287,2537,18165,11004,2240,9825,0,2241,0,1464,15654
141060,Mitchell,17431,1985,11356,23181,1214,5248,0,1585,2,1436,26594
141092,Paterson,26766,2809,20530,18343,4022,3706,14,1957,4,1510,23324
141437,Warringah,18610,1173,3170,12828,783,2777,0,914,17312,582,19757
142615,North Sydney,19170,2114,8513,14568,422,3193,0,695,10151,1383,19757
144025,Mackellar,17104,1676,3201,16873,1063,2380,0,1236,16240,228,24119


In [66]:
newgeometry = final_table.simplify(tolerance=0.0005)
newfinal_table = gpd.GeoDataFrame(final_table,geometry=newgeometry)

In [67]:
# Save the final_table as a shapefile
newfinal_table.to_file(f"{path}/datasa1/2022-federal-election-votes-sa1.shp")

C:\Users\patri\AppData\Local\Temp\ipykernel_31120\3703719027.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  newfinal_table.to_file(f"{path}/datasa1/2022-federal-election-votes-sa1.shp")


In [18]:
sa1s = gpd.read_file(f'{path}/datasa1/2022-federal-election-votes-sa1.shp')
sa1s.head
output = sa1s.explore(column="DivisionNm", cmap="tab20")
outfile = f'{path}/docs/index.html'
output.save(outfile)